In [16]:
!pip uninstall -y langchain langchain-core langchain-community langchain-google-genai
!pip install langchain==0.1.0 langchain-community==0.0.10 langchain-core==0.1.8 langchain-google-genai==0.0.6 google-search-results
#!pip install -q "requests<2.32" youtube-search-python --upgrade
!pip install -q yt-dlp

Found existing installation: langchain 0.1.0
Uninstalling langchain-0.1.0:
  Successfully uninstalled langchain-0.1.0
Found existing installation: langchain-core 0.1.8
Uninstalling langchain-core-0.1.8:
  Successfully uninstalled langchain-core-0.1.8
Found existing installation: langchain-community 0.0.10
Uninstalling langchain-community-0.0.10:
  Successfully uninstalled langchain-community-0.0.10
Found existing installation: langchain-google-genai 0.0.6
Uninstalling langchain-google-genai-0.0.6:
  Successfully uninstalled langchain-google-genai-0.0.6
  Using cached langchain-0.1.0-py3-none-any.whl.metadata (13 kB)
  Using cached langchain_community-0.0.10-py3-none-any.whl.metadata (7.3 kB)
  Using cached langchain_core-0.1.8-py3-none-any.whl.metadata (4.0 kB)
  Using cached langchain_google_genai-0.0.6-py3-none-any.whl.metadata (2.7 kB)
Using cached langchain-0.1.0-py3-none-any.whl (797 kB)
Using cached langchain_community-0.0.10-py3-none-any.whl (1.5 MB)
Using cached langchain_core-

In [20]:
import os
# os.environ.pop("HTTP_PROXY", None)
# os.environ.pop("HTTPS_PROXY", None)
from langchain.tools import tool
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import initialize_agent, AgentType
import wikipedia
import yt_dlp
#from youtubesearchpython import VideosSearch
#from youtubesearchpython import VideosSearch

In [ ]:
# --- CONFIGURATION ---
# # 1. Google Gemini Key
os.environ["GOOGLE_API_KEY"] = ""

# # 2. SerpAPI Key (Get free at serpapi.com)
os.environ["SERPAPI_API_KEY"] = ""
# --- CHECK KEYS ---
if not os.environ.get("GOOGLE_API_KEY"):
    print("⚠️ STOP: Please enter your Google API Key!")
else:
    try:
        # --- DEFINE TOOLS ---
      

        @tool
        def search_wiki(query: str):
            """to get a 1-sentence cultural summary of 'Hyderabadi Biryani'"""
            print(f"    🔎 [Search Tool] Wiki: '{query}'")
            return wikipedia.summary(query,sentences=1)

        # 2. Calculator
        @tool
        def calculator(expression: str):
            """Useful for calculating math expressions."""
            print(f"    🧮 [Math Tool] Calculating: '{expression}'")
            try:
                return str(eval(expression))
            except:
                return "Error"
        # 2. Youtube
        @tool
        def youtube(query: str):
            
            
            """Search YouTube and return top video's title and views"""
            # videos = VideosSearch(query, limit=1).result()
            # video = videos["result"][0]
        
            # title = video["title"]
            # views = video["viewCount"]["text"]
        
            # return f"Title: {title}, Views: {views}"
            with yt_dlp.YoutubeDL({"quiet": True}) as ydl:
                info = ydl.extract_info(f"ytsearch1:{query}", download=False)
                video = info["entries"][0]
        
                return f"Title: {video['title']}, Views: {video['view_count']}"

        

        tools = [search_wiki, calculator,youtube]

        # --- INITIALIZE AGENT ---
        llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash") # Using generic pro model for compatibility

        # The "Zero Shot ReAct" agent is the standard "Think-Act-Observe" loop
        agent_executor = initialize_agent(
            tools=tools,
            llm=llm,
            agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
            verbose=True,
            handle_parsing_errors=True # Helps if the LLM makes a formatting mistake
        )

        # --- RUN TASK ---
        print("\n--- 🤖 REAL AGENT TASK STARTED ---")
        query = """
            You are a content analyst.
            
            1. Get a 1-sentence cultural summary of Hyderabadi Biryani from Wikipedia.
            2. Find the top YouTube video on Hyderabadi Biryani and extract its view count.
            3. Calculate estimated earnings assuming CPM is ₹150 per 1000 views.
            4. Present the final earnings in Lakhs or Crores.
            
            Be precise."""
        # In older versions, we use .run() instead of .invoke()
        response = agent_executor.run(query)

        print(f"\n--- 🏁 FINAL ANSWER ---\n{response}")

    except Exception as e:
        print(f"\n❌ Error: {e}")



--- 🤖 REAL AGENT TASK STARTED ---


> Entering new AgentExecutor chain...
Action: search_wiki
Action Input: Hyderabadi Biryani    🔎 [Search Tool] Wiki: 'Hyderabadi Biryani'

Observation: Hyderabadi biryani (also known as Hyderabadi dum biryani) is a style of biryani originating from Hyderabad, India made with basmati rice and meat (mostly goat meat).
Thought:Action: youtube
Action Input: Hyderabadi Biryani


Observation: Title: The ORIGINAL Hyderabadi Biryani | Bawarchi Biryani | Alpha Hotel | Hyderabad | Kunal Vijayakar, Views: 1943457
Thought:Action: calculator
Action Input: (1943457 / 1000) * 150    🧮 [Math Tool] Calculating: '(1943457 / 1000) * 150'

Observation: 291518.55
Thought:I now know the final answer
Final Answer: 
1. **Cultural Summary:** Hyderabadi biryani (also known as Hyderabadi dum biryani) is a style of biryani originating from Hyderabad, India made with basmati rice and meat (mostly goat meat).
2. **Top YouTube Video Views:** The top YouTube video on Hyderabadi Biryani, titled "The ORIGINAL Hyderabadi Biryani | Bawarchi Biryani | Alpha Hotel | Hyderabad | Kunal Vijayakar", has 1,943,457 views.
3. **Estimated Earnings:** The estimated earnings, assuming a CPM of ₹150 per 1000 views, are ₹2,91,518.55.
4. **Earnings in Lakhs/Crores:** The estimated earnings are 2.92 Lakhs.

> Finished chain.

--- 🏁 FINAL ANSWER ---
1. **Cultural Summary:** Hyderabadi biryani (also known a